In [1]:
import geopandas as gpd
from fractopo import branches_and_nodes
from pathlib import Path
from tval import trace_validator, trace_builder
import matplotlib.pyplot as plt
from shapely.geometry import Polygon

/home/nialov/.local/share/virtualenvs/fractopo-YImANkM_/lib/python3.8/site-packages/geopandas/_compat.py:84: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.8.1-CAPI-1.13.3). Conversions between both will be slow.
  warnings.warn(
4650-DEBUG-(private) matplotlib data path: /home/nialov/.local/share/virtualenvs/fractopo-YImANkM_/lib/python3.8/site-packages/matplotlib/mpl-data
4650-DEBUG-matplotlib data path: /home/nialov/.local/share/virtualenvs/fractopo-YImANkM_/lib/python3.8/site-packages/matplotlib/mpl-data
4650-DEBUG-CONFIGDIR=/home/nialov/.config/matplotlib
4650-DEBUG-matplotlib version 3.3.2
4650-DEBUG-interactive is False
4650-DEBUG-platform is linux
4650-DEBUG-loaded modules: ['sys', 'builtins', '_frozen_importlib', '_imp', '_warnings', '_frozen_importlib_external', '_io', 'marshal', 'posix', '_thread', '_weakref', 'time', 'zipimport', '_codecs', 'codecs', 'encodings.aliases', 'encodings', 'encodings.ut

In [2]:
import logging
logging.basicConfig(level=logging.WARNING)

In [3]:
layer = "Inkoo_Infinity_Lineaments_traces"
filepath = Path("tests/sample_data/Inkoo_traces.gpkg")
gdf = gpd.read_file(filepath, layer=layer)
snap_threshold = 0.001


4650-DEBUG-Entering env context: <fiona.env.Env object at 0x7fe19d6b22b0>
4650-DEBUG-Starting outermost env
4650-DEBUG-No GDAL environment exists
4650-DEBUG-New GDAL environment <fiona._env.GDALEnv object at 0x7fe19ce44fd0> created
4650-DEBUG-Logging error handler pushed.
4650-DEBUG-All drivers registered.
4650-DEBUG-GDAL_DATA found in environment: '/home/nialov/.local/share/virtualenvs/fractopo-YImANkM_/lib/python3.8/site-packages/fiona/gdal_data'.
4650-DEBUG-PROJ data files are available at built-in paths
4650-DEBUG-Started GDALEnv <fiona._env.GDALEnv object at 0x7fe19ce44fd0>.
4650-DEBUG-Updated existing <fiona._env.GDALEnv object at 0x7fe19ce44fd0> with options {}
4650-DEBUG-Entered env context: <fiona.env.Env object at 0x7fe19d6b22b0>
4650-DEBUG-Layer has no coordinate system
4650-DEBUG-Projection not found (cogr_crs was NULL)
4650-DEBUG-Layer has no coordinate system
4650-DEBUG-Projection not found (cogr_crs was NULL)
4650-DEBUG-OLC_FASTSETNEXTBYINDEX: 0
4650-DEBUG-OLC_FASTFEATUR

In [4]:
#gdf = gpd.GeoDataFrame({"geometry": trace_builder.make_valid_traces()})

In [5]:
det_nodes, _ = trace_validator.BaseValidator.determine_nodes(gdf)
det_nodes = gpd.GeoSeries(det_nodes)

In [6]:
nodes_rem_id = branches_and_nodes.remove_identical_sindex(det_nodes, snap_threshold)

In [7]:
node_identities = branches_and_nodes.get_node_identities(gdf.geometry, nodes_rem_id, snap_threshold=snap_threshold)

In [ ]:
nodes_f_y_nodes = branches_and_nodes.find_y_nodes_and_snap_em(gdf.geometry, nodes_rem_id, node_identities, snap_threshold)

In [ ]:
branches = branches_and_nodes.split_traces_to_branches(gdf.geometry, nodes_f_y_nodes, node_identities)

In [ ]:
branch_identities = branches_and_nodes.get_branch_identities(branches, nodes_f_y_nodes, node_identities, snap_threshold)

In [ ]:
branch_gdf = gpd.GeoDataFrame({"geometry": branches, "Type": branch_identities})
node_gdf = gpd.GeoDataFrame({"geometry": nodes_f_y_nodes, "Type": node_identities})

In [ ]:
fig, ax = plt.subplots(figsize=(9, 9))
branch_gdf.plot(ax=ax, column="Type", label="Type", legend=True)
node_gdf.plot(ax=ax, column="Type", label="Type", legend=True, zorder=10)

In [ ]:
branch_gdf

In [ ]:
fig, ax = plt.subplots(figsize=(9, 9))
branch_gdf.loc[branch_gdf.Type == "E-E"].plot(ax=ax, column="Type", label="Type", legend=True)
node_gdf.loc[node_gdf.intersects(
    Polygon(branch_gdf.loc[branch_gdf.Type == "E-E"].iloc[0].geometry.bounds)
)].plot(ax=ax, column="Type", label="Type", legend=True, zorder=10)

In [ ]:
branch_gdf.loc[branch_gdf.Type == "E-E"].iloc[0].geometry.bounds

In [ ]:
Polygon([
    (350000, 6655377)
])